# Data Acquisition and Formatting

This notebook provides the workflow for aquiring and formatting the data needed to run the Pandemic Network Model. To run this notebook, the following are assumed:
- Data are saved in a project folder (e.g., Google Drive root H:/Project Folder/)
- Cloned the Pandemic GitHub repository (git clone https://github.com/ncsu-landscape-dynamics/Pandemic_Model.git)
- Notebook was launched from the notebook folder of the cloned repo
- Already have the following data available or downloaded:
    - Koppen-Geiger Climate Classification raster (e.g., Beck_KG_V1_present_0p083.tif from http://koeppen-geiger.vu-wien.ac.at/data)
    - Phytosanitary capacity (data frame with country name and ISO3 code, estimate/index of phytosanitary capacity)
    - Binary host map raster
    - File with the following environmental variables:
        - DATA_PATH (file path to data folder that will contain original data, formatted model input data, and model output data)
        - COMTRADE_AUTH_KEY (API key to query and download data from the UN Comtrade Database)

## Imports

In [ ]:
import sys
import os
import subprocess
import math
import glob
import json
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.enums import Resampling
from rasterstats import zonal_stats
import dotenv 
from scipy.spatial import distance
from functools import reduce

In [ ]:
# If notebook was launched from notebook folder of the clone GitHub
# repository, then set working directory to level above 
# (e.g., /Pandemic_Model)
os.chdir('../')
print(os.getcwd())

In [ ]:
from Data.Comtrade.get_comtrade import query_comtrade
from pandemic.generate_trade_forecasts import simple_trade_forecast
from pandemic.helpers import distance_between
from pandemic.ecological_calculations import create_climate_similarities_matrix

## Set Environmental Variables and Paths

In [ ]:
repo_path = os.getcwd() # should be one folder above launch directory
print(repo_path)

In [ ]:
# Provide file paths to where .env file is saved and load file
project_path = str(input())

In [ ]:
env_file = os.path.join(project_path, '.env') 
dotenv.load_dotenv(env_file)

In [ ]:
# Root project data folder
data_path = os.getenv('DATA_PATH')

# Path to formatted model inputs
input_dir = data_path + "inputs/"

# Path to save outputs
out_dir = data_path + "outputs/"

In [ ]:
# Confirm all directories exist:
dir_list = [data_path, input_dir, out_dir]
for d in dir_list:
    if not os.path.exists(d):
        print(f"ERROR:\t{d} does not exist, creating directory...")
        os.makedirs(d)
    else:
        print(f"Confirmed:\t{d}")

In [ ]:
# Add model data intput and output paths to environment file for 
# use in run_model.ipynb
print(dotenv.set_key(env_file, "INPUT_PATH", input_dir))
print(dotenv.set_key(env_file, "OUTPUT_PATH", out_dir))

## Country Data

In [ ]:
countries_geo_path = "H:/Shared drives/APHIS  Projects/Pandemic/Data/Country_list_shapefile/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp"

In [ ]:
countries_gdf = gpd.read_file(countries_geo_path)
countries_gdf.iloc[136,4] = 'Macao'
countries_gdf.iloc[169,4] = 'Réunion'
countries_gdf.iloc[17,4] = 'Myanmar'
countries_gdf.iloc[245, 4] = 'Saint Barthelemy'

## Koppen-Geiger Climate Classification by Country
This step only needs to be run once. 

It creates a data frame consisting of countries as rows, climate
classification codes as columns, and percent area in each country
as values. 

In [ ]:
# Read Koppen raster
koppen_rast = rasterio.open("H:/Shared drives/Data/Raster/Global/Beck_KoppenClimate/Beck_KG_V1_present_0p083.tif")
koppen_arr = koppen_rast.read(1)
koppen_arr.shape

In [ ]:
# Read Human Influence Index layer (already resampled to match Koppen raster)
hii = rasterio.open("H:/Shared drives/APHIS  Projects/Pandemic/Data/land_use/human_influence_index/hii_v2_resamp.tif")
hii_arr = hii.read(1)
hii_arr = hii_arr.astype('float64')
hii_arr.shape

In [ ]:
# Create mask to exclude areas with values below the threshold 
# from the % area calculations of climate similaritiy and 
# host availability 
threshold_val = 16

# Mask values less than threshold (water is already 255)
hii_arr[hii_arr < threshold_val] = 0
hii_arr[hii_arr == 255] = 0

# Keep areas greater than threshold
hii_arr[hii_arr >= threshold_val] = 1
hii_arr[hii_arr == 0] = np.nan

In [ ]:
# Mask selected areas
koppen_masked = hii_arr * koppen_arr

In [ ]:
# Calculate zonal statistics
affine = koppen_rast.transform
stats = zonal_stats(countries_gdf, koppen_masked, categorical=True, affine = affine)

In [ ]:
# Add % of each climate classification to countries geodataframe
koppen_df = countries_gdf.loc[:, ['ISO3', 'NAME']]
koppen_df['koppen_stats'] = stats
koppen_df = pd.concat([koppen_df, koppen_df['koppen_stats'].apply(pd.Series)], axis=1).fillna(0)
pix_ct = pd.DataFrame(koppen_df.sum(axis=1))
cat_pct = koppen_df.iloc[:,3:].div(pix_ct[0], axis=0, fill_value=None)

kg_codes = pd.read_csv("H:/Shared drives/Data/Raster/Global/Beck_KoppenClimate/KGcodes.csv")
cat_pct = cat_pct.iloc[:,1:]
cat_pct.columns = list(kg_codes["let"])
koppen_df = pd.concat([koppen_df.iloc[:,0:2], cat_pct], axis=1)

koppen_df = koppen_df.fillna(0)
koppen_df.drop(["NAME"], axis=1, inplace=True)
koppen_df = koppen_df.set_index("ISO3")

In [ ]:
koppen_df.to_csv(input_dir + '/koppen_hiiMask' + str(threshold_val) + '_wTWN.csv', sep = ',')

In [ ]:
# If loading from file:
#koppen_df = pd.read_csv(f"{input_dir}/koppen_hiiMask' + str(threshold_val) + 'wTWN.csv", index_col=0)

In [ ]:
koppen_df.head()

## UN Comtrade Data

This step only needs to be run once per aggregation (e.g., monthly, annual, start year,
commodity code). 

It queries the UN Comtrade API to download data based on the first year
of interest, end year (inclusive), commodity codes, frequency (e.g., monthly, annual), 
and unit value (e.g., value in dollars or net weight). Data are saved as csvs by HS code
and time step. 

In [ ]:
comtrade_auth_key = os.getenv("COMTRADE_AUTH_KEY")
start_commodity = 6801
end_commodity = 6804
start_year = 2000
end_year = 2019
temporal_res = 'M'

In [ ]:
query_comtrade(
    # model_inputs_dir=input_dir,
    model_inputs_dir=input_dir + '/comtrade_wTWN',
    auth_code=comtrade_auth_key,
    start_code=start_commodity,
    end_code=end_commodity,
    start_year=start_year,
    end_year=end_year,
    temporal_res=temporal_res,
    crosswalk_path="H:/Shared drives/APHIS  Projects/Pandemic/Data/un_to_iso_wTWN.csv")

### Adjust Trade Data for Inflation

Get the Consumer Price Index from the US Bureau of Labor Statistics

Series CUUR0000SA0L1E - All items less food and energy in U.S. city average, all urban consumers, not seasonally adjusted

Other CPI series are available. See more information here: https://www.bls.gov/cpi/

### TODO: Write workflow for annual data

In [ ]:
# Download CPI data
# API allows 10 years per request, divide year requests into lists of 10 or less
years = list(range(start_year, end_year + 1))
year_subsets = [years[x:x+10] for x in range(0, len(years), 10)]
cpi_series = "CUUR0000SA0L1E"

cpi_list = []
for subset in year_subsets:
    print(f"Downloading CPI for {str(subset[0])}-{str(subset[-1])}...")
    headers = {"Content-type": "application/json"}
    data = json.dumps(
        {"seriesid": [cpi_series], "startyear": str(subset[0]), "endyear": str(subset[-1])}
    )
    p = requests.post(
        "https://api.bls.gov/publicAPI/v1/timeseries/data/", data=data, headers=headers
    )
    json_data = json.loads(p.text)
    json_data = json_data["Results"]["series"][0]

    for ts in json_data["data"]:
        year = ts["year"]
        period = ts["period"]
        value = ts["value"]
        cpi_list.append([year, period, value])

cpi_df = pd.DataFrame(cpi_list, columns=['year', 'period', 'cpi'])
cpi_df["period"] = cpi_df["period"].str.lstrip('M')
cpi_df["ts"] = cpi_df["year"] + cpi_df["period"]
cpi_df = cpi_df.set_index("ts")
print(f"CPI for {len(cpi_df)} timesteps downloaded.")

In [ ]:
# Choose baseline year to convert current (nominal) $ to
base_ts = "201901"
cpi_base = cpi_df.loc[base_ts,"cpi"]

In [ ]:
if temporal_res == 'M':
    file_list = glob.glob(input_dir + '/comtrade_wTWN/monthly/*/*.csv')
    print(f"Converting current $ to {base_ts} $ for {len(file_list)} files...")
    for file in file_list:
        file_name = file.split("\\")[-1]
        ts = file_name[5:11]
        cpi_ts = cpi_df.loc[ts,"cpi"]
        out_dir = input_dir + '/comtrade_wTWN/monthly_adjusted/' + file_name[:4]
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        trade = pd.read_csv(file,index_col=0)
        trade_adjusted = (trade  * (float(cpi_base) / 100)) / (float(cpi_ts) / 100)
        trade_adjusted.to_csv(out_dir + "/" + file_name)
    print(f"Adjusted trade values saved at {input_dir}/comtrade_wTWN/monthly_adjusted/")

### Aggregated Multiple Commodities

This step only needs to be run once per download and if running the model based
on all commodities of interest (as opposed to by each commodity) is planned. 

In [ ]:
if str(start_commodity)[:2] == str(end_commodity)[:2]:
    code_pre = str(start_commodity)[:2]
else:
    code_pre = '-agg'
print(code_pre)

In [ ]:
# If trade data are monthly
if temporal_res == 'M':
    file_list = glob.glob(f'{input_dir}/comtrade_wTWN/monthly_adjusted/{start_commodity}/{str(start_commodity)}*.csv')
    date_list = []
    for fn in file_list:
        date = os.path.splitext(fn)[0].split('_')[-1]
        date_list.append(date)

    date_list_unique = list(set(date_list))
    date_list_unique.sort()
    
    out_path = f'{input_dir}/comtrade_wTWN/monthly_agg/{str(start_commodity)}-{str(end_commodity)}/'
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    for d in date_list_unique:
        d_file_list = glob.glob(input_dir + f'/comtrade_wTWN/monthly_adjusted/*/*{d}*.csv')
        print(f'{d}: {len(d_file_list)}')
        dfs = [pd.read_csv(f, sep = ",", header= 0, index_col=0, encoding='latin1') for f in d_file_list]
        all_com = reduce(pd.DataFrame.add, dfs)
        all_com.to_csv(out_path + f"HS{code_pre}_trades_{d}.csv")
        
# # If trade data are annual
# if temporal_res == 'A':
#     out_path = input_dir + f'/comtrade_wTWN/annual_agg/{str(start_commodity)}-{str(end_commodity)}/'
#     if not os.path.exists(out_path):
#         os.makedirs(out_path)

#     year_range = list(range(start_year, end_year + 1, 1))
#     for d in year_range:
#         d_file_list = glob.glob(input_dir + f'/comtrade_wTWN/annual/*/*{d}.csv')
#         print(f'{d}: {len(d_file_list)}')
#         dfs = [pd.read_csv(f, sep = ",", header= 0, index_col=0, encoding='latin1') for f in d_file_list]
#         all_com = reduce(pd.DataFrame.add, dfs)
#         all_com.to_csv(out_path + f"HS{code_pre}_trades_{d}.csv")

### Create Trade Forecast

This step only needs to be run once per aggregation. It is a simple sampling of
historical trade data to be used as predictions of future trade values.

TO DO: Add ability to include a percent change (e.g., 1% increase) by year or time
horizon. 

In [ ]:
hist_trade_dir = input_dir + "/comtrade_wTWN/monthly_agg/6801-6804"
forecast_dir = input_dir + "/comtrade_wTWN/trade_forecast/monthly_agg/6801-6804"
start_forecast_year = 202001
number_historical_years = 5
number_forecast_years = 10
random_seed = 47

In [ ]:
simple_trade_forecast(
    data_dir=input_dir + '/comtrade_wTWN',
    output_dir=forecast_dir,
    start_forecast_date=start_forecast_year,
    num_yrs_historical=number_historical_years,
    num_yrs_forecast=number_forecast_years,
    hist_data_dir=hist_trade_dir,
    random_seed =random_seed
)

## Phytosanitary Capacity Data
Incorporates an estimate of phytosanitary capacity for each country. Currently the model 
is using the proactive value from:

Early, R., Bradley, B., Dukes, J. et al. Global threats from invasive alien species in the twenty-first century and national response capacities. Nat Commun 7, 12485 (2016). https://doi-org.prox.lib.ncsu.edu/10.1038/ncomms12485

In [ ]:
phyto_path = "H:/Shared drives/APHIS  Projects/Pandemic/Data/phytosanitary_capacity/phytosanitary_capacity_iso3.csv"
phyto_df = pd.read_csv(phyto_path, index_col=0)

In [ ]:
phyto_df= phyto_df[["proactive", "ISO3", "UN"]]
phyto_df = phyto_df.rename(columns={"proactive": "Phytosanitary Capacity"})

In [ ]:
phyto_df.shape

In [ ]:
## NEED TO UPDATE THIS IN add_iso3_phyto.py 
phyto_df = phyto_df.append(
    {
        'ISO3': "USA", 
        'Phytosanitary Capacity': 3.0, 
        'UN': 840
    }, ignore_index=True)
phyto_df

## Host
The step only needs to be run once. 

Using a binary host map, calculate the percent area in each country with
probable presence of host.

In [ ]:
# Read bindary host raster
host = rasterio.open('H:/Shared drives/Data/Raster/Global/toh_global_tr.tif')
host_arr = host.read(1)

# Set large negative values to 0
host_arr[host_arr < 0.0001] = 0

In [ ]:
host_arr = np.append(host_arr, np.zeros([hii_arr.shape[0] - host_arr.shape[0], host_arr.shape[1]]), axis=0)
host_masked = host_arr * hii_arr

In [ ]:
# Calculate zonal statistics
affine = host.transform
stats = zonal_stats(countries_gdf, host_masked, categorical=True, affine = affine)

In [ ]:
# Create host dataframe with country identifiers, results from zonal stats, and calculate host percent area
host_df = countries_gdf.loc[:, ['ISO3', 'NAME']]
host_df['host_stats'] = stats
host_df = pd.concat([host_df, host_df['host_stats'].apply(pd.Series)], axis=1).fillna(0)
host_df['Host Percent Area'] = (host_df[1.0] / (host_df[0.0] + host_df[1.0])).fillna(0)
host_df.iloc[136,1] = 'Macao'
host_df.iloc[169,1] = 'Réunion'
host_df.iloc[17,1] = 'Myanmar'
host_df.iloc[245, 1] = 'Saint Barthelemy'
host_df.head()

In [ ]:
host_df.to_csv(input_dir + '/host_hiiMask' + str(threshold_val) + '_wTWN.csv', sep = ',')

In [ ]:
# If loading from file:
#host_df = pd.read_csv(input_dir + '/host_hiiMask' + str(threshold_val) + 'wTWN.csv', sep=',')
#host_df.drop(['Unnamed: 0', 'host_stats', '0.0', '1.0'], axis=1, inplace=True)
#host_df.head()

## Create final countries dataframe
Merge country attributes with host percent area, climate classificaiton percent area, 
and phytosanitary capacity estimates. Filter and order dataframe to match countries 
with trade data available. 

In [ ]:
# Merge koppen, host, and phytosanitary attributes with countries geodataframe
countries_gdf = countries_gdf.merge(koppen_df, on='ISO3')
countries_gdf = countries_gdf.merge(host_df[['ISO3', 'Host Percent Area']], how='left', on='ISO3')
countries_gdf = countries_gdf.merge(phyto_df, how="left", on="ISO3", suffixes=[None, "_y"])
countries_gdf.head()

In [ ]:
# Get unique values of phytosanitary capacity for rescaling 
unique_keys = list(countries_gdf['Phytosanitary Capacity'].unique())
unique_keys.sort()

In [ ]:
# Rescale input phytosanitary capacity values using specified
# minimum and maxmimum values

# Minimum phytosanitary capacity value when rescaled
scaled_min = 0.3

# Maximum phytosanitary capacity value when rescaled
scaled_max = 0.8

phyto_dict = {}

for i in unique_keys:
    if np.isnan(i):
        phyto_dict[i] = 0
    else:
        if np.isnan(unique_keys).any():
            increments = len(unique_keys) - 1
        else:
            increments = len(unique_keys)
        
        scale_diff = scaled_max - scaled_min
        phyto_dict[i] = round((scale_diff / increments) * unique_keys.index(i), 2) + scaled_min

phyto_dict

In [ ]:
countries_gdf["Phytosanitary Capacity"] = countries_gdf["Phytosanitary Capacity"].replace(phyto_dict) 

In [ ]:
countries_gdf.set_index('ISO3', inplace=True)
countries_gdf.head()

In [ ]:
# Read example trade matrix to identify which countries are in the geodataframe but not
# in the trade data
example_trade = pd.read_csv(input_dir + '/comtrade_wTWN/monthly_agg/6801-6804/HS68_trades_201001.csv', header=0, index_col=0, encoding='latin-1')
country_set = set(countries_gdf.index.values)
trade_set = set(example_trade.index.values)

In [ ]:
print('# ISO3 codes in countries geopackage:\t', len(country_set))
print('# ISO3 code matches:\t\t\t', len(trade_set.intersection(country_set)))

print('Which countries are in the TRADE data but NOT the COUNTRIES geopackage')
miss_country = trade_set - country_set
print('\n', miss_country)

print('Which countries are in the COUNTRIES geopackage but NOT the TRADE data:')
miss_trade = country_set - trade_set 
print('\n', miss_trade)

IMN: Isle of Man
PRI: Puerto Rico
ALA: Aland Islands
LIE: Liechtenstein
MTQ: Martinique
GUF: French Guiana
MAF: Saint-Martin (French part)
TWN: Taiwan
JEY: Jersey
MCO: Monaco
GGY: Guernsey
GLP: Guadeloupe
REU: Réunion
VIR: US Virgin Islands
BVT: Bouvet Island
SJM: Svalbard and Jan Mayen Islands

In [ ]:
# Remove countries from the geodataframe that do not have trade data
countries_filtered = countries_gdf.drop(miss_trade, axis=0)

In [ ]:
# Reorder the countries geodataframe rows to match the 
# trade index order
index_list = list(example_trade.index.values)
countries_filtered_reindex = countries_filtered.loc[index_list, :]

In [ ]:
countries_filtered_reindex.reset_index(inplace=True)
countries_filtered_reindex.head()

In [ ]:
# Save filtered and reindexed countries data with climate, host %s, and phyto values
countries_path = input_dir + "/countries_slf_hiiMask" + str(threshold_val) + "_wTWN.gpkg"
countries_filtered_reindex.to_file(countries_path, driver='GPKG')

In [ ]:
# Save full country data frame path to .env file
print(dotenv.set_key(env_file, "COUNTRIES_PATH", countries_path))

## Create Distance Matrix
Calculate the distance between each origin-destination country pair. Save 
as matrix array. 

In [ ]:
distances = distance_between(countries_filtered_reindex)

In [ ]:
np.save(input_dir + '/distance_matrix_wTWN.npy', distances)

## Create Climate Simiarities Matrix
Calculate the similarity between each origin-destination country pair. Save as matrix array.

In [ ]:
# Create an n x n array of climate similarity calculations
climate_similarities = create_climate_similarities_matrix(
    array_template=example_trade, countries=countries_filtered_reindex
)

In [ ]:
np.save(input_dir + '/climate_similarities_hiiMask' + str(threshold_val) + '_wTWN.npy', climate_similarities)